In [ ]:
import numpy as np
import pandas as pd
import math
import os
import cv2
import matplotlib.pyplot as plt

In [ ]:
IMAGE_DIM=50

In [ ]:
DONT_INVERT_IMAGE_DIR = "training-e"
def read_image(path):
    img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    # Resizing the image
    img = cv2.resize(img,(IMAGE_DIM, IMAGE_DIM))
    # Displaying the image
    # plt.imshow(img, cmap='gray')
    # plt.show()
    # print(img.shape)
    img = np.array(img)
    img = img.astype('float32')
    # Inverting the image
    if DONT_INVERT_IMAGE_DIR not in path:
        img = 255 - img
    # dilation
    kernel = np.ones((2,2), np.uint8)
    img = cv2.dilate(img, kernel, iterations=1)
    # plt.imshow(img, cmap='gray')
    # plt.show()
    # print(img.shape)
    img /= 255
    # reshaping the image
    img = img.reshape(IMAGE_DIM, IMAGE_DIM, 1) # 1 for grayscale
    return img
# read_image('./NumtaDB/training-e/e00002.png')

In [ ]:
# https://pytorch.org/tutorials/beginner/basics/data_tutorial.html#creating-a-custom-dataset-for-your-files
class CustomImageDataset:
    def __init__(self, annotations_file, img_dir, transform=None, target_transform=None):
        self.img_labels = pd.read_csv(annotations_file)
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0])
        image = read_image(img_path)
        label = self.img_labels.iloc[idx, 3] # 3 is the column index of the label
        # if self.transform:
        #     image = self.transform(image)
        # if self.target_transform:
        #     label = self.target_transform(label)
        return image, label

In [ ]:
import numpy as np
import os
import cv2
import pandas as pd

class CustomDataLoader:
    def __init__(self, dataset, batch_size=32, shuffle=False):
        self.dataset = dataset
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.current_idx = 0

    def __iter__(self):
        return self

    def __next__(self):
        if self.current_idx >= len(self.dataset):
            raise StopIteration
        
        # Get the next batch. If the batch size is larger than the number of
        # remaining elements, just return the remaining elements.
        if self.current_idx + self.batch_size > len(self.dataset):
            batch = [self.dataset[i] for i in range(self.current_idx, len(self.dataset))]
            self.current_idx = len(self.dataset)
        else:
            batch = [self.dataset[i] for i in range(self.current_idx, self.current_idx + self.batch_size)]
            self.current_idx += self.batch_size
        

        if self.shuffle:
            np.random.shuffle(batch)

        images, labels = zip(*batch)
        images = np.stack(images)
        labels = np.stack(labels)

        return images, labels

annotations_file = "./NumtaDB/training-d.csv"
img_dir = "./NumtaDB/training-d"
dataset = CustomImageDataset(annotations_file, img_dir)
dataloader = CustomDataLoader(dataset, batch_size=64, shuffle=False)

for images, labels in dataloader:
    print(images.shape, labels.shape)

In [ ]:
img = cv2.imread('a00000.png', cv2.IMREAD_GRAYSCALE)
print(img.shape)
# Resizing the image
img = cv2.resize(img,(IMAGE_DIM, IMAGE_DIM))
# Displaying the image
plt.imshow(img, cmap='gray')
plt.show()
print(img.shape)

In [ ]:
# !jupyter nbconvert --to script analyzer.ipynb
# !python analyzer.py

In [ ]:
import numpy as np
arr = np.array([[[20, 2, 3], [4, 15, 6]], [[7, 18, 9], [110, 11, 12]]])
flat_idx = np.argmax(arr, axis=(2))
ij_idx = np.unravel_index(flat_idx, arr.shape[:2])
print(arr)
print(flat_idx)
print(ij_idx)